In [4]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns; sns.set_theme()
import math
# %matplotlib widget

In [75]:
# check values in info.txt
header = ['dens', 'vx', 'vy', 'prs', 'cs'] 
xcell = 64
xbeg = -0.25
xend = 0.25
ycell = 1024*2
ybeg = 0.
yend = 16.
Pstar = 24.
Ptr = 9.0

final= 3.
steps = 100.


In [76]:
class Snapshot:
    def __init__(self, icell = xcell, jcell = ycell, ibeg = xbeg, jbeg = ybeg, iend = xend, jend = yend, ptr = Ptr, pstar = Pstar, tend = final, steps = steps):
        self.icell = icell
        self.jcell = jcell
        self.ibeg = ibeg
        self.jbeg = jbeg
        self.iend = iend
        self.jend = jend
        self.ptr = ptr
        self.pstar = pstar
        self.data = {}
        self.Xgrid, self.Ygrid = np.meshgrid(np.linspace(ibeg, iend, icell), np.linspace(jbeg, jend, jcell))
        self.out = ""
#         self.step = 0
        self.time = ""
        self.tend = tend
        self.tstep = steps
        return
        
        
    def set_data(self, step):
        idx = 0
        header = ['dens', 'vx', 'vy', 'prs', 'cs', 'eps', 'E']
        self.out = str(step).zfill(4)
#         self.step = step
        self.time = str(round(step * 3.336 * self.tend/self.tstep,2))
        filename = '../data/NS.' + self.out + '.dat'
        
        for var in header:
            self.data[var] = np.loadtxt(filename , unpack=True, usecols=[idx]).reshape(self.jcell,self.icell)
            idx += 1
        return
    
    def get_data(self):
        return self.data

            
    def plot(self, var, jmax ,title = 'Plot'):
        output = '../imagens/NS_' + var + '_' + self.out + '.png'
        fig, ax = plt.subplots(1,1)
        plt.style.use('seaborn-white')
        cp = ax.contourf(self.Xgrid, self.Ygrid, self.data[var], cmap=  plt.cm.coolwarm ) #'viridis' )#'gist_heat')
#         cs = ax.contour(self.Xgrid, self.Ygrid, self.data[var], levels = [20.,self.ptr] ,
#                  colors=('k',),linestyles=('-',),linewidths=(0.7,))#'gist_heat')
#         fig.colorbar(cp) # Add a colorbar to a plot
        ax.set_title(title + ' time: ' + self.time + '$\mu$s')
        ax.set_xlabel('x (Km)')
        ax.set_ylabel('y (Km)')
        ax.set_xlim(self.ibeg, self.iend)
#         ax.set_ylim(self.jbeg, self.jend)
        ax.set_ylim(self.jbeg, jmax)
#         ax.axes.set_aspect('equal')
#         ax.axis('scaled')
#         if (var == 'prs'):
#             norm = mpl.colors.TwoSlopeNorm(vmin=self.data[var].min(), vmax=self.data[var].max(), vcenter = self.ptr)
#         elif(var == 'dens'):
#             rho = 939. * ( 1.333 * (self.ptr + 120.) )**0.75 / (1.772 * 197.33**0.75)
#             norm = mpl.colors.TwoSlopeNorm(vmin=self.data[var].min(), vmax=self.data[var].max(), vcenter = rho)
        norm = mpl.colors.Normalize(vmin=self.data[var].min(), vmax= self.data[var].max())

        sm = plt.cm.ScalarMappable(norm=norm, cmap = cp.cmap)
        sm.set_array([])
        fig.colorbar(sm, ticks=cp.levels) # , label='$MeV\cdot fm^{-3}$')

#         plt.show()
        plt.savefig(output)

        plt.close('all')

        return

    def velocity(self, var, jmax, title='Plot'):        
        output = '../imagens/NS_' + var + '_vel_' + self.out + '.png'
        fig, ax = plt.subplots(1,1)
        plt.style.use('seaborn-white')
        cp = ax.contourf(self.Xgrid, self.Ygrid, self.data[var], cmap='viridis' )
#         fig.colorbar(cp) # Add a colorbar to a plot
        ax.set_title(title)
        ax.set_xlabel('x (Km)')
        ax.set_ylabel('y (Km)')
        ax.set_xlim(self.ibeg, self.iend)
        ax.set_ylim(self.jbeg, jmax)
        norm = mpl.colors.Normalize(vmin=self.data[var].min(), vmax=self.data[var].max())
        sm = plt.cm.ScalarMappable(norm=norm, cmap = cp.cmap)
        sm.set_array([])
        fig.colorbar(sm, ticks=cp.levels) # , label='$MeV\cdot fm^{-3}$')

        speed = np.sqrt(self.data['vx']**2 + self.data['vy']**2)
        UN = self.data['vx'] /speed
        VN = self.data['vy'] /speed
        ax.quiver(self.Xgrid, self.Ygrid, UN, VN, 
           color='Teal', 
           headlength=10)
        
    def plot4var(self, pos):

        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, sharex=True)
        plt.style.use('seaborn-white')
        fig.suptitle('t = ' + self.time + '$\mu$s')
        fig.tight_layout()
        
        axis_y = np.linspace(self.jbeg, self.jend, self.jcell)

        ax1.plot(axis_y, self.data['dens'][:,pos])
        ax1.set_title('Density $ [MeV\cdot fm^{-3}]$ ')
    #     ax1.set(ylabel='$ MeV\cdot fm^{-3}$')            

        ax2.plot(axis_y, self.data['prs'][:,pos])
        ax2.set_title('Pressure $ [MeV\cdot fm^{-3}]$')
#         ax2.plot(axis_y, np.full(self.jcell, self.ptr), label = "Ptr")
        ax2.plot(axis_y, np.full(self.jcell, self.pstar), label = "Pstar")
    #     ax2.set(ylabel='$ MeV\cdot fm^{-3}$')  

#         ax3.plot(axis_y, self.data['vy'][:,pos])
#         ax3.set_title('Velocity in y direction $[c = 1]$')
#     #     ax3.set(ylabel='$ c = 1$')
#         ax4.plot(axis_y, self.data['vx'][:,pos])
#         ax4.set_title('Velocity in x direction $[c = 1]$')
    #     ax4.set(ylabel='$ c = 1$')
    
        ax3.plot(axis_y, self.data['eps'][:,pos])
        ax3.set_title('internal energy')
    #     ax3.set(ylabel='$ c = 1$')
        ax4.plot(axis_y, self.data['E'][:,pos])
        ax4.set_title('Total Energy')
    
        #limites dos eixos y
        ax1.set_xlim(self.jbeg,self.jend)
    #     ax1.set_ylim(-50, 750)
    #     ax2.set_ylim(-2., 100)
    #     ax3.set_ylim(-0.25, 0.25)

        filename = '../imagens/NS.' + self.out + '.png'
        fig.savefig(filename)
        plt.close()




     

In [77]:
pic = Snapshot()

In [90]:
pic.set_data(40)

In [91]:
pic.plot4var(32)

In [92]:
pic.plot('prs',4, title = 'Pressure $MeV\cdot fm^{-3}$')
pic.plot('dens',4, title = 'Density $MeV\cdot fm^{-3}$')

In [ ]:
pic = Snapshot()
for step in range(0,101):
    pic.set_data(step)
    pic.plot4var(16)
    pic.plot('prs',16, title = 'Pressure $MeV\cdot fm^{-3}$')
    pic.plot('dens',16, title = 'Density $MeV\cdot fm^{-3}$')
    

In [ ]:
pic.plot('prs',16, title = 'Pressure $MeV\cdot fm^{-3}$')

In [ ]:
pic.plot('dens',16, title = 'Density $MeV\cdot fm^{-3}$')

In [ ]:
pic.velocity('prs',2)

In [ ]:
pic.get_data(50)
pic.plot('prs',3, title = 'Density $MeV\cdot fm^{-3}$')

In [ ]:
def get_data4(step, pos, jcell = ycell, icell = xcell):
    data = np.linspace(0, 16, ycell)
    i = str(step).zfill(4)
    filename = '../data/NS.' + i + '.dat'
    var = 0
#     pos = math.floor(icell/2)
    for var in [0,1,2,3]: # densidade, vx vy, e pressao
        prim = np.loadtxt(filename , unpack=True, usecols=[var]).reshape(jcell,icell)
#         print(prim[:,math.floor(icell/2)].shape)
        data = np.column_stack((data, prim[:,pos]))
#         print(data.shape)
    return data

def plot4var(data, step):
    
    time = str(round(step * 0.0336 * 30.,2))
    
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, sharex=True)
    fig.suptitle('t = ' + time + '$\mu$s')
    fig.tight_layout()
    i = 1
    for ax in [ax1, ax4, ax3, ax2]:
        ax.plot(data[:,0], data[:,i])
        i += 1
    ax2.plot(data[:,0], np.full(len(data[:,0]), 24))
    ax1.set_title('Density $ [MeV\cdot fm^{-3}]$ ')
#     ax1.set(ylabel='$ MeV\cdot fm^{-3}$')
    ax2.set_title('Pressure $ [MeV\cdot fm^{-3}]$')
#     ax2.set(ylabel='$ MeV\cdot fm^{-3}$')
    ax3.set_title('Velocity in y direction $[c = 1]$')
#     ax3.set(ylabel='$ c = 1$')
    ax4.set_title('Velocity in x direction $[c = 1]$')
#     ax4.set(ylabel='$ c = 1$')
    #limites dos eixos y
    ax1.set_xlim(0,16)
#     ax1.set_ylim(-50, 750)
#     ax2.set_ylim(-2., 100)
#     ax3.set_ylim(-0.25, 0.25)

    i = str(step).zfill(4)
    filename = '../imagens/NS.' + i + '.png'
    fig.savefig(filename)






In [ ]:
step = 100
pos = 16
plot4var(get_data4(step, pos),step)

In [ ]:
for step in range(0,101):
    plot4var(get_data4(step),step)

In [ ]:
test.shape

In [ ]:
for step in range(30,41):
    plot4var(get_data4(step),step)